# Retroactive changes
* Notebooks from now on will be on their own folders

This makes is so much easier to keep track of dependencies, and we also resolve the issue of IfCntk overriding the local <code>main.group.fsx</code> path.

# Preparing workspace

Our new dependency for this notebook is Newtonsoft.Json, since json is so easy to use from javascript.

In [ ]:
#load "Paket.fsx"
 
Paket.Dependencies.Install """
framework: netstandard2.0
generate_load_scripts: true
storage: none
source https://nuget.org/api/v2
nuget CNTK.CPUOnly
nuget Newtonsoft.Json
nuget MathNet.Numerics
nuget MathNet.Numerics.FSharp
"""

In [3]:
#r "netstandard"
#r @"..\bin\Cntk.Core.Managed-2.6.dll"
#load "../Graphviz/.paket/load/main.group.fsx"

In [4]:
open System
open System.IO

Environment.GetEnvironmentVariable("PATH")
|> fun path -> sprintf "%s%c%s" path (Path.PathSeparator) (Path.GetFullPath("bin"))
|> fun path -> Environment.SetEnvironmentVariable("PATH", path)

open CNTK
DeviceDescriptor.UseDefaultDevice().Type
|> printfn "Congratulations, you are using CNTK for: %A" 

Congratulations, you are using CNTK for: CPU


In [ ]:
let device = CNTK.DeviceDescriptor.CPUDevice
let dataType = CNTK.DataType.Float
let initialization = CNTKLib.GlorotUniformInitializer(1.0)
let input_dim, num_output_classes = 2,2

#load "../fsx/CntkHelpers.fsx"
open CntkHelpers

# Model building

Let's create a simple classifier with one hidden layer of 50 nodes that outputs to a softmax function:

In [56]:
let input = Variable.InputVariable(shape [|input_dim|], dataType, "Features")
let label = Variable.InputVariable(shape [|num_output_classes|], dataType, "Labels")
let z = 
    fullyConnectedClassifierNet input [50] num_output_classes CNTKLib.Sigmoid
    |> (Var>>CNTKLib.Softmax)    

# The Search for Nodes

The only way on offer for retrieving the model's consituent components seems to be retrieving <code>Function</code> objects by name, using <code>Function.FindByName/FindAllByName</code>. Maintaining a naming scheme while building a model in CNTK inmy opinion not especially expedient at the best of times, and a downright pain if you are training to implement functional idioms.

In addition, if you are trying to catch a glimpse of the internal workings of a CNTK model it's likely because you had no hand in creating it in the first place. A way to blindly extract network achitecture seems to be necessary.

In [57]:
/// Graph search for CNTK.Function objects.
/// We use the Input & Output lists in a Function
/// to find other Function objects by checking
/// the Variable's Owner property.
/// <remarks> CNTK helper function </remarks>
let decomposeFunction (root: Function) = 
    let visited = System.Collections.Generic.Dictionary<string, Function>()            

    let rec expand (f: Function) = 
        match visited.ContainsKey(f.Uid) with
        | true -> Seq.empty
        | false -> 
            visited.Add(f.Uid, f)
            seq {
                yield f
                yield! 
                    seq { yield! f.Inputs
                          yield! f.Outputs }
                    |> Seq.map (fun v -> v.Owner)      
                    |> Seq.filter (not<<isNull)
                    |> Seq.collect expand
            }        

    Array.ofSeq (expand root)

Recursively accumulating connected variables and checking the Owner property seems to be an easy way

In [62]:
open System.Collections.Generic

let (|IsEnumerable|IsDescribable|IsPrimitive|) (t: Type) = 
    if  t <> typeof<string> && 
        (typeof< IEnumerable<_> >).IsAssignableFrom(t) 
    then IsEnumerable
    else if t.GetMethods() 
            |> Array.exists (fun meth -> meth.Name = "AsString") 
    then IsDescribable
    else IsPrimitive

let asString item =
    if isNull item then ""
    else
        match item.GetType() with
        | IsDescribable ->     
            item.GetType()
                .GetMethod("AsString", Array.empty)
                .Invoke(item, Array.empty).ToString()
        | _ -> item.ToString()

In [63]:
open System.Reflection

let describeNode (item: obj) =
    [|
        yield KeyValuePair("NodeType", item.GetType().Name)
        yield KeyValuePair("AsString", item |> asString)
        yield!
            item.GetType().GetProperties()
            |> Seq.map
                (fun prop ->
                    match prop.PropertyType with
                    | IsEnumerable -> 
                        prop.Name, 
                          (prop.GetValue(item) :?> IEnumerable<_>) 
                          |> function 
                          | list when list |> Seq.isEmpty -> "[]" 
                          | list -> 
                              list 
                              |> Seq.map (asString)                         
                              |> Seq.reduce (sprintf "%s, %s")
                    | IsDescribable -> prop.Name, prop.GetValue(item) |> asString
                    | IsPrimitive -> 
                        prop.Name,
                            try prop.GetValue(item) |> asString
                            with ex -> sprintf "%s" ex.Message)
            |> Seq.map (fun (k,v) -> KeyValuePair(k,v))
    |]

In [81]:
// https://github.com/Microsoft/CNTK/blob/master/bindings/python/cntk/logging/graph.py

let extractGraphVizDotNotation (f: Function) = 
        let varText (v:Variable) = (if String.IsNullOrEmpty v.Name then v.Uid else v.Name)// + "\\n" + v.Shape.AsString()
        let funText (f: Function) = if String.IsNullOrEmpty f.Name then f.Uid else f.Name
    
        let varLabel (v: Variable) = sprintf "%s [label=\"%s\"];" v.Uid (varText v)
        let funLabel (f: Function) = sprintf "%s [label=\"%s\"];" f.Uid (funText f)

        let varShape (v: Variable) =
            match v with
            | _ when v.IsInput -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsOutput -> sprintf "%s [shape=invhouse, color=gray];" v.Uid
            | _ when v.IsPlaceholder -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsParameter -> sprintf "%s [shape=diamond, color=green];" v.Uid
            | _ when v.IsConstant -> sprintf "%s [shape=rectangle, color=lightblue];" v.Uid
            | _ -> sprintf "%s [shape=circle, color=purple];" v.Uid

        let funShape (f: Function) = 
            match f with 
            | _ when f.IsComposite -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, peripheries=2];" f.Uid
            | _ when f.IsPrimitive -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, size=0.6];" f.Uid
            | _ -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=4];" f.Uid

        let varEdges (f: Function) (v: Variable) = 
            let inputIndex = f.Inputs |> Seq.map (fun v -> v.Uid) |> Set
            let outputIndex = f.Outputs |> Seq.map (fun v -> v.Uid) |> Set
            match inputIndex.Contains(v.Uid), outputIndex.Contains(v.Uid) with 
            | true, _ when v.IsParameter -> sprintf "%s -> %s [label=\"input param\"];" v.Uid f.Uid |> Some
            | _, true when v.IsParameter -> sprintf "%s -> %s [label=\"output param\"];" f.Uid v.Uid|> Some
            | true, _ -> sprintf "%s -> %s [label=input];" v.Uid f.Uid|> Some 
            | _, true -> 
                if f.Outputs.Count = 1 && f.Output.Uid = v.Uid then
                    sprintf "%s -> %s [label=output];" f.Uid v.Owner.Output.Uid |> Some
                else sprintf "%s -> %s [label=output];" f.Uid v.Uid|> Some            
            | _ -> None
        
        let varOwner (v: Variable) =
            match v.Owner with
            | null -> None
            | _ -> sprintf "%s -> %s [style=\"dotted\"];" v.Owner.Uid v.Uid |> Some

        let vars = Seq.append f.Inputs f.Outputs
        let funs = seq { 
                yield f
                yield f.RootFunction;
                yield! vars |> Seq.map (fun v -> v.Owner) |> Seq.filter (isNull>>not) 
            } 

        seq {        
            if f.Uid <> f.RootFunction.Uid then yield sprintf "%s -> %s [label=\"root function\"];" f.RootFunction.Uid f.Uid
            yield! vars |> Seq.map varShape
            yield! vars |> Seq.map varLabel
            yield! vars |> Seq.map (varEdges f) |> Seq.choose id
            yield! vars |> Seq.map varOwner |> Seq.choose id
            yield! funs |> Seq.map funLabel 
            yield! funs |> Seq.map funShape
        } |> Seq.distinct


<div style="width: 100%; height: 700px; max-height: 700px; overflow-y: scroll; overflow-x: hidden; border: solid lightblue 1px; border-radius: 15px; position: relative">
    <div id="graph" style="width: 100%; height: 100%;"></div>
    <div id="info" style="width: 350px; max-width: 350px; position: absolute; top: 0; right: 0; background-color: white"></div>
    <div style="display: flex;position: absolute; top: 10px; left: 10px; align-items: center;">
        <span>Graphviz engine:</span>
        <select id="engineSelect" class="form-control" style="width: 150px">
            <option>dot</option>
            <option>circo</option>
            <option>fdp</option>
            <option>neato</option>
            <option>osage</option>
            <option>patchwork</option>
            <option>twopi</option>
        </select>
    <div>
</div>

In [10]:
@"<script src='../../../d3-jupyter/dist/bundle.js'></script>" |> Util.Html |> Display

In [11]:
let initGraph infoPath graphPath = 
    sprintf "<script>$(document).trigger('INIT_D3', ['%s','%s']);</script>" infoPath graphPath
    |> Util.Html
    
let renderDot engine dotNotation jsonInfo = 
    sprintf "<script>$(document).trigger('RENDER_GRAPH', [`%s`,`%s`, '%s']);</script>" dotNotation jsonInfo engine
    |> Util.Html
    
let renderSeries engine (digraphs : string[]) = 
    digraphs
    |> Array.reduce(sprintf "%s','%s")
    |> fun graphs -> graphs,engine
    ||> sprintf "<script>$(document).trigger('RENDER_SERIES', [['%s'],'%s']);</script>" 
    |> Util.Html   

In [69]:
module Engine =    
    let dot = "dot"
    let circo = "circo"
    let fdp = "fdp"
    let neato = "neato"
    let osage = "osage"
    let patchwork = "patchwork"
    let twopi = "twopi"

In [82]:
open Newtonsoft.Json

let createGraphWithInfo engine (model: Function) =
    let funcs = model |> decomposeFunction |> Array.filter (fun f -> not f.IsComposite)
    let vars = 
        funcs 
        |> Array.collect(fun f -> [|yield! f.Inputs; yield! f.Outputs|])
        |> Array.distinctBy (fun v -> v.Uid)
    
    let nodeInfo = 
        [|  yield "{"
            yield
                funcs 
                |> Array.map 
                    (fun f -> sprintf "\"%s\": %s" (f.Uid) (f |> describeNode |> JsonConvert.SerializeObject))                            
                |> Array.reduce(sprintf "%s,\n%s")
            yield ","
            yield
                vars 
                |> Array.map 
                    (fun v -> sprintf "\"%s\": %s" (v.Uid) (v |> describeNode |> JsonConvert.SerializeObject))                            
                |> Array.reduce(sprintf "%s,\n%s")
            yield "}" |]
        |> Array.reduce(sprintf "%s\n%s")
        |> fun text -> text.Trim();
    
    let dotNotation =
        funcs
        |> Array.collect (extractGraphVizDotNotation>>Array.ofSeq)
        |> Array.distinct 
        |> Array.reduce(sprintf "%s\n%s")
        |> sprintf "digraph { %s }"
            
    initGraph "#info" "#graph" |> Display
    renderDot engine dotNotation nodeInfo |> Display
    
createGraphWithInfo Engine.dot z

In [42]:
let displayGradualGraph engine (model: Function) =
    
    let callJs engine graphs = 
        (graphs,engine)
        ||> sprintf "<script>$(document).trigger('RENDER_SERIES', [['%s'], '%s']);</script>"
        |> Util.Html  
        
    initGraph "#info" "#gradualGraph" |> Display
    
    model
    |> decomposeFunction 
    //|> Array.filter (fun f -> not f.IsComposite)
    |> Array.map (extractGraphVizDotNotation>>Seq.reduce(sprintf "%s %s")) 
    |> Array.rev
    |> Array.scan (sprintf "%s %s") ""
    |> Array.map (sprintf "digraph { %s }")
    |> Array.reduce (sprintf "%s','%s")
    |> callJs engine    
    |> Display
            
displayGradualGraph Engine.dot z

In [37]:
let classifier = z

In [50]:
displayGradualGraph Engine.dot classifier

<div id="gradualGraph" style="width: 100%; min-height: 700px; border: solid aliceblue 1px">
</div>

In [ ]:
type Tensor = 
| F of Function 
| V of Variable
with 
    member x.Uid = match x with F f -> f.Uid | V v -> v.Uid
    member x.AsString = match x with F f -> f.AsString() | V v -> v.AsString()

let decomposeTensor (root: Tensor) = 
        
        let visited = Dictionary<string, Tensor>()            

        let rec search (f: Tensor) = 
            visited.Add(f.Uid, f)
        
            seq {   
                match f with
                | F f' -> 
                    yield! f'.Inputs |> Seq.map V
                    yield! f'.Outputs |> Seq.map V
                    yield  f'.RootFunction |> F 
                | V v' -> 
                    yield f
                    if v'.Owner |> isNull |> not then 
                        yield v'.Owner |> F
            }
            |> Seq.filter (fun f -> visited.ContainsKey(f.Uid) |> not)
            |> Seq.iter search

        search root
    
        visited

In [ ]:
"""<script>$(document).on("NODE_CLICKED", 
    (e,d,i,n)=>{
        console.log(e,d,i,n);
});</script>""" |> Util.Html

In [ ]:
"../../../d3-jupyter/dist/bundle.js" |> System.IO.Path.GetFullPath